In [4]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from zoneinfo import ZoneInfo

# -------------------------
# LOAD DATA
# -------------------------
file_path = "googleplaystore.csv"  # change path accordingly
df = pd.read_csv(file_path)

# -------------------------
# DATA CLEANING
# -------------------------
# Clean Installs
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Clean Reviews
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")

# Convert Size to MB (optional if needed)
def size_to_mb(x):
    if isinstance(x, str):
        if "M" in x:
            return float(x.replace("M",""))
        elif "k" in x:
            return float(x.replace("k","")) / 1024
    return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# Last Updated to datetime
df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")

# -------------------------
# FILTERS
# -------------------------
filtered = df[
    (~df["App"].str.startswith(("x","y","z"), na=False)) &
    (~df["App"].str.contains("S", na=False)) &
    (df["Reviews"] > 500) &
    (df["Category"].str.startswith(("E","C","B"), na=False))
]

# -------------------------
# MONTHLY TOTAL INSTALLS
# -------------------------
filtered["YearMonth"] = filtered["Last Updated"].dt.to_period("M")
monthly_installs = (
    filtered.groupby(["YearMonth", "Category"])["Installs"]
    .sum()
    .reset_index()
)

monthly_installs["YearMonth"] = monthly_installs["YearMonth"].astype(str)

# Pivot for plotting
pivot_df = monthly_installs.pivot(index="YearMonth", columns="Category", values="Installs").fillna(0)

# -------------------------
# LEGEND TRANSLATION
# -------------------------
translations = {
    "Beauty": "सौंदर्य",        # Hindi
    "Business": "வணிகம்",      # Tamil
    "Dating": "Partnersuche"   # German
}

pivot_df.rename(columns=translations, inplace=True)

# -------------------------
# TIME CONDITION (6PM – 9PM IST)
# -------------------------
current_time = datetime.now(ZoneInfo("Asia/Kolkata"))
current_hour = current_time.hour

print("Current IST Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

if 18 <= current_hour < 21 and not pivot_df.empty:

    fig = go.Figure()

    for category in pivot_df.columns:
        fig.add_trace(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[category],
            mode='lines+markers',
            name=category
        ))

        # Highlight periods with >20% MoM growth
        growth = pivot_df[category].pct_change()
        for i, val in enumerate(growth):
            if val > 0.2:
                fig.add_vrect(
                    x0=pivot_df.index[i-1], x1=pivot_df.index[i],
                    fillcolor="yellow", opacity=0.3, line_width=0
                )

    fig.update_layout(
        title="Monthly Total Installs by Category\n(Visible Only Between 6 PM IST and 9 PM IST)",
        xaxis_title="Month",
        yaxis_title="Total Installs",
        template="plotly_white"
    )

    fig.show()

else:
    print("⚠ Dashboard Locked")
    print("Time series chart visible ONLY between 6 PM IST and 9 PM IST.")


C:\Users\GOD\AppData\Local\Temp\ipykernel_14776\2027536818.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["YearMonth"] = filtered["Last Updated"].dt.to_period("M")


Current IST Time: 2026-02-16 10:12:29
⚠ Dashboard Locked
Time series chart visible ONLY between 6 PM IST and 9 PM IST.
